# Creating the Timer class

Note: The underscore (_) prefix of ._start_time is a Python convention. It signals that ._start_time is an internal attribute that users of the Timer class shouldn’t manipulate.

In [1]:
import time

class TimerError(Exception):
    """A custom exception used to report errors in use of Timer class"""

class Timer:
    def __init__(self):
        self._start_time = None

    def start(self):
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()

    def stop(self):
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None
        print(f"Elapsed time: {elapsed_time:0.4f} seconds")

# Adding adaptable text

Note: If you want to use an f-string to specify .text, then you need to use double curly braces to escape the curly braces that the actual elapsed time will replace.

One example would be f"Finished {task} in {{:0.4f}} seconds". If the value of task is "reading", then this f-string would be evaluated as "Finished reading in {:0.4f} seconds".

In [2]:
import time

class TimerError(Exception):
    """A custom exception used to report errors in use of Timer class"""

class Timer:
    def __init__(self, text="Elapsed time: {:0.4f} seconds"):
        self._start_time = None
        self.text = text

    def start(self):
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()

    def stop(self):
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None
        print(self.text.format(elapsed_time))
        
t = Timer(text="You waited {:.1f} seconds")
t.start()
time.sleep(1)
t.stop()  # A few seconds later

You waited 1.0 seconds


# Adding logging functions

In [3]:
import logging

class Timer:
    def __init__(
        self,                                 
        text="Elapsed time: {:0.4f} seconds",
        logger=print #New
    ):
        self._start_time = None
        self.text = text
        self.logger = logger #New

    # Other methods are unchanged
    def start(self):
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()

    def stop(self):
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None

        if self.logger: #New If you pass None, it wont print.
            self.logger(self.text.format(elapsed_time)) 

        return elapsed_time
    
t = Timer(logger=logging.warning)
t.start()
t.stop()

2.1000000000270802e-05

# Adding time measurements

- Class variables can be accessed either directly on the class or through an instance of the class

- Used to keep track of multiple timers

In [4]:
import logging

class Timer:
    
    timers = {} #NEW: class variable so all instances of Timer will share it
    
    def __init__(
        self,
        name = None,
        text="Elapsed time: {:0.4f} seconds",
        logger=print 
    ):
        
        self._start_time = None
        self.name = name
        self.text = text
        self.logger = logger 
        
    # Add new named timers to dictionary of timers
        if name:
            self.timers.setdefault(name,0)
    
    # Other methods are unchanged
    def start(self):
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()

    def stop(self):
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None

        if self.logger: #If you pass None, it wont print.
            self.logger(self.text.format(elapsed_time)) 
            
        if self.name:
            self.timers[self.name] += elapsed_time

        return elapsed_time

In [5]:
# Accessing through Class
Timer.timers

#Accessing thru instance
t = Timer()
t.timers

{}

In [6]:
t = Timer("accumulate")
t.start()
t.stop()
print(Timer.timers)

Elapsed time: 0.0000 seconds
{'accumulate': 2.2299999999475517e-05}


# Changing timer class to dataclass

- The @dataclass decorator defines Timer as a data class.

- The special ClassVar annotation is necessary for data classes to specify that .timers is a class variable.

- .name, .text, and .logger will be defined as attributes on Timer, whose values can be specified when creating Timer instances. They all have the given default values.

- Recall that ._start_time is a special attribute that’s used to keep track of the state of the Python timer, but it should be hidden from the user. Using dataclasses.field(), you say that ._start_time should be removed from .__init__() and the representation of Timer.

- You can use the special .__post_init__() method for any initialization that you need to do apart from setting the instance attributes. Here, you use it to add named timers to .timers.


In [7]:
t = Timer()
t

In [8]:
import logging
import time
from dataclasses import dataclass, field
from typing import Any, ClassVar

@dataclass
class Timer:
    
#New: Replaces .__init__() --------------------------------------
    
    
    timers: ClassVar = {} #Necessary to indicate classvariable
    name: Any = None
    text: Any = "Elapsed time: {:0.4f} seconds"
    logger: Any = print
    _start_time: Any = field(default=None, init=False, repr=False) #

    def __post_init__(self):
        """Initialization: add timer to dict of timers"""
        if self.name:
            self.timers.setdefault(self.name, 0)
        
        self._start_time = None
        self.name = name
        self.text = text
        self.logger = logger 
#-------------------------------------------------------------

    # Add new named timers to dictionary of timers
        if name:
            self.timers.setdefault(name,0)
    
    # Other methods are unchanged
    def start(self):
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()

    def stop(self):
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None

        if self.logger: #If you pass None, it wont print.
            self.logger(self.text.format(elapsed_time)) 
            
        if self.name:
            self.timers[self.name] += elapsed_time

        return elapsed_time

In [9]:
print(1)

1


# Touch ups

- Readability: Your code will read more naturally if you carefully choose class and method names.
- Consistency: Your code will be easier to use if you encapsulate properties and behaviors into attributes and methods.
- Flexibility: Your code will be reusable if you use attributes with default values instead of hard-coded values.

In [10]:
from dataclasses import dataclass, field
import time
from typing import Callable, ClassVar, Dict, Optional

class TimerError(Exception):
    """A custom exception used to report errors in use of Timer class"""

@dataclass
class Timer:
    timers: ClassVar[Dict[str, float]] = {}
    name: Optional[str] = None
    text: str = "Elapsed time: {:0.4f} seconds"
    logger: Optional[Callable[[str], None]] = print
    _start_time: Optional[float] = field(default=None, init=False, repr=False)

    def __post_init__(self) -> None:
        """Add timer to dict of timers after initialization"""
        if self.name is not None:
            self.timers.setdefault(self.name, 0)

    def start(self) -> None:
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()

    def stop(self) -> float:
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        # Calculate elapsed time
        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None

        # Report elapsed time
        if self.logger:
            self.logger(self.text.format(elapsed_time))
        if self.name:
            self.timers[self.name] += elapsed_time

        return elapsed_time

# Python Timer Context Manager

.__enter__() called before running code

.__exit__() called after

.__exit__() takes three arguments: exc_type, exc_value, and exc_tb used for error handling.

exit is called before exception/error is raised in the code.

In [11]:
# greeter.py

class Greeter:
    def __init__(self, name):
        self.name = name

    def __enter__(self):
        print(f"Hello {self.name}")
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        print(f"See you later, {self.name}")

In [12]:
with Greeter("Akshay"):
    print("Doing stuff ...")

Hello Akshay
Doing stuff ...
See you later, Akshay


In [13]:
class TimerError(Exception):
    """A custom exception used to report errors in use of Timer class"""

@dataclass
class Timer:
    timers: ClassVar[Dict[str, float]] = {}
    name: Optional[str] = None
    text: str = "Elapsed time: {:0.4f} seconds"
    logger: Optional[Callable[[str], None]] = print
    _start_time: Optional[float] = field(default=None, init=False, repr=False)

    def __post_init__(self) -> None:
        """Add timer to dict of timers after initialization"""
        if self.name is not None:
            self.timers.setdefault(self.name, 0)

    def start(self) -> None:
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()

    def stop(self) -> float:
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        # Calculate elapsed time
        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None

        # Report elapsed time
        if self.logger:
            self.logger(self.text.format(elapsed_time))
        if self.name:
            self.timers[self.name] += elapsed_time
        return elapsed_time
            
    def __enter__(self):
        """Start a new timer as a context manager"""
        self.start()
        return self
    
    def __exit__(self, *exc_info):
        """Stop the context manager timer"""
        self.stop()



In [14]:
with Timer():
    time.sleep(0.7)

Elapsed time: 0.7005 seconds


# Decorators
A decorator is a function that wraps another function to modify its behavior. This technique is possible because functions are first-class objects in Python. In other words, functions can be assigned to variables and used as arguments to other functions, just like any other object. This gives you a lot of flexibility and is the basis for several of Python’s most powerful features.

First, note that turn_off() is just a regular function. What makes this a decorator is that it takes a function as its only argument and returns a function. You can use turn_off() to modify other functions, like this:

In [15]:
#Random decorator that does nothing
def turn_off(func):
    return lambda *args, **kwargs: None

The line print = turn_off(print) decorates the print statement with the turn_off() decorator. Effectively, it replaces print() with lambda *args, **kwargs: None returned by turn_off().

In [20]:
print("Hello")


print = turn_off(print)
print("Hush")
# Nothing is printed

# Inner Functions
An inner function is a function that’s defined inside another function. One common use of inner functions is to create function factories:

multiplier() is an inner function, defined inside create_multiplier(). Note that you have access to factor inside multiplier(), while multiplier() isn’t defined outside create_multiplier():

In [16]:
def create_multiplier(factor):
    def multiplier(num):
        return factor * num
    return multiplier

In [17]:
# multiplier
# NameError: name 'multiplier' is not defined


Instead you use create_multiplier() to create new multiplier functions, each based on a different factor:

In [18]:
double = create_multiplier(factor=2)
double(3)

6

Similarly, you can use inner functions to create decorators. Remember, a decorator is a function that returns a function:

triple() is a decorator, because it’s a function that expects a function, func(), as its only argument and returns another function, wrapper_triple(). Note the structure of triple() itself:

The parameters are *args and **kwargs, which collect whichever positional and keyword arguments you pass to the function. This gives you the flexibility to use triple() on any function.

In [17]:
def triple(func):
    def wrapper_triple(*args, **kwargs):
        print(f"Tripled {func.__name__!r}")
        value = func(*args, **kwargs)
        return value * 3
    return wrapper_triple

In [18]:
def knock():
    return "Penny! "
knock = triple(knock)
result = knock()
result

'Penny! Penny! Penny! '

In [19]:
@triple
def knock():
    return "Penny! "
result = knock()
result

'Penny! Penny! Penny! '

@triple decorates knock(), which is then replaced by the wrapper_triple() inner function, as the output above confirms. This will also replace the name, docstring, and other metadata. Often, this won’t have much effect, but it can make introspection difficult.

Sometimes, decorated functions must have correct metadata. @functools.wraps fixes exactly this issue:

In [22]:
import functools

def triple(func):
    @functools.wraps(func)
    def wrapper_triple(*args, **kwargs):
        print(f"Tripled {func.__name__!r}")
        value = func(*args, **kwargs)
        return value * 3
    return wrapper_triple

In [23]:
def hi(func):
    def wrapper_hi(*args,**kwargs):
        print("hello")
        value = func(*args, **kwargs)
        return value * 3
    return wrapper_hi

@hi
def knock():
    return "Penny! "

knock()

'Penny! Penny! Penny! '

In [24]:
def make_pretty(func):
    def inner():
        print("I got decorated")
        func()
    return inner


def ordinary():
    print("I am ordinary")

print('hello')

In [33]:
print('hello')

In [25]:
@triple
def knock():
    return "Penny! "

knock()

'Penny! Penny! Penny! '

In [26]:
#Template 
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator

# Applying decorators to timer class

In [27]:
import functools
import time

def timer(func):
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        tic = time.perf_counter()
        value = func(*args, **kwargs)
        toc = time.perf_counter()
        elapsed_time = toc - tic
        print(f"Elapsed time: {elapsed_time:0.4f} seconds")
        return value
    return wrapper_timer


#This decorator doenst work for some reason

In [28]:
@timer
def say_hi():
    return 1
say_hi()

1

# Converting your class to callables (functions)

In [29]:
def sq(num):
    return num ** 2
print(sq(4))

class Sqer:
    def __call__(self,num):
        return num ** 2
square = Sqer()
square(4)

16

In [30]:
class TimerError(Exception):
    """A custom exception used to report errors in use of Timer class"""

@dataclass
class Timer:
    timers: ClassVar[Dict[str, float]] = {}
    name: Optional[str] = None
    text: str = "Elapsed time: {:0.4f} seconds"
    logger: Optional[Callable[[str], None]] = print
    _start_time: Optional[float] = field(default=None, init=False, repr=False)

    def __post_init__(self) -> None:
        """Add timer to dict of timers after initialization"""
        if self.name is not None:
            self.timers.setdefault(self.name, 0)

    def start(self) -> None:
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()

    def stop(self) -> float:
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        # Calculate elapsed time
        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None

        # Report elapsed time
        if self.logger:
            self.logger(self.text.format(elapsed_time))
        if self.name:
            self.timers[self.name] += elapsed_time
        return elapsed_time
            
    def __enter__(self):
        """Start a new timer as a context manager"""
        self.start()
        return self
    
    def __exit__(self, *exc_info):
        """Stop the context manager timer"""
        self.stop()
        
    def __call__(self, func):
        """Support using Timer as a decorator"""
        @functools.wraps(func)
        def wrapper_timer(*args, **kwargs):
            with self:
                return func(*args, **kwargs)

        return wrapper_timer



.__call__() uses the fact that Timer is already a context manager to take advantage of the conveniences that you’ve already defined there. Make sure you also import functools at the top of timer.py.

You can now use Timer as a decorator:

In [31]:
@Timer(text="Downloaded the tutorial in {:.2f} seconds")
def knock():
    return "Penny! "
knock()

'Penny! '

In [32]:
import time
from contextlib import ContextDecorator
from dataclasses import dataclass, field
from typing import Any, Callable, ClassVar, Dict, Optional

class TimerError(Exception):
    """A custom exception used to report errors in use of Timer class"""

@dataclass
class Timer(ContextDecorator):
    """Time your code using a class, context manager, or decorator"""

    timers: ClassVar[Dict[str, float]] = {}
    name: Optional[str] = None
    text: str = "Elapsed time: {:0.4f} seconds"
    logger: Optional[Callable[[str], None]] = print
    _start_time: Optional[float] = field(default=None, init=False, repr=False)

    def __post_init__(self) -> None:
        """Initialization: add timer to dict of timers"""
        if self.name:
            self.timers.setdefault(self.name, 0)

    def start(self) -> None:
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()

    def stop(self) -> float:
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        # Calculate elapsed time
        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None

        # Report elapsed time
        if self.logger:
            self.logger(self.text.format(elapsed_time))
        if self.name:
            self.timers[self.name] += elapsed_time

        return elapsed_time

    def __enter__(self) -> "Timer":
        """Start a new timer as a context manager"""
        self.start()
        return self

    def __exit__(self, *exc_info: Any) -> None:
        """Stop the context manager timer"""
        self.stop()